In [1]:
from __future__ import division, print_function
from pprint import pprint
# from raymond hettinger modern python dicts talk - pycon oregon may 2017

keys = 'guido sarah barry rachel tim'.split()
values1 = 'blue orange green yellow red'.split()
values2 = 'austin denver tuscon reno portland'.split()
values3 = 'apple banna orange pear peach'.split()
hashes = list(map(abs, map(hash, keys)))
entries = list(zip(hashes, keys, values1))
comb_entries = list(zip(hashes, keys, values1, values2, values3))

In [7]:
entries

[(3205703081059114744, 'guido', 'blue'),
 (4293989514786313987, 'sarah', 'orange'),
 (8459490058338865399, 'barry', 'green'),
 (6432004217604261546, 'rachel', 'yellow'),
 (3926032199644776547, 'tim', 'red')]

In [2]:
# db approach - challenge is it is a linear search
def database_linear_search():
    pprint(list(zip(keys, values1, values2, values3)))
    
database_linear_search()

[('guido', 'blue', 'austin', 'apple'),
 ('sarah', 'orange', 'denver', 'banna'),
 ('barry', 'green', 'tuscon', 'orange'),
 ('rachel', 'yellow', 'reno', 'pear'),
 ('tim', 'red', 'portland', 'peach')]


In [4]:
# lisp approach
def association_lists():
    pprint([
        list(zip(keys, values1)),
        list(zip(keys, values2)),
        list(zip(keys, values3))
    ])
association_lists()

[[('guido', 'blue'),
  ('sarah', 'orange'),
  ('barry', 'green'),
  ('rachel', 'yellow'),
  ('tim', 'red')],
 [('guido', 'austin'),
  ('sarah', 'denver'),
  ('barry', 'tuscon'),
  ('rachel', 'reno'),
  ('tim', 'portland')],
 [('guido', 'apple'),
  ('sarah', 'banna'),
  ('barry', 'orange'),
  ('rachel', 'pear'),
  ('tim', 'peach')]]


In [14]:
# separate chaining - reduce linear search
entries_local= list(zip(keys, values1))

# not sure why this isn't working in 3.6   (it was written for 2.7)
def separate_chaining(n):
    buckets = [[] for i in range(n)]
    for pair in entries_local:
        key, value = pair
        i = hash(key) % n
        buckets[i].append(pair)
    pprint(buckets)

separate_chaining(2)
#separate_chaining(4)
#separate_chaining(8)

[[('guido', 'blue'), ('rachel', 'yellow')],
 [('sarah', 'orange'), ('barry', 'green'), ('tim', 'red')]]


In [15]:
# four people found on 1 probe, barry would take 2
separate_chaining(4)
# 8 you still have a single collision, lots of empty spaces
separate_chaining(8)


[[('guido', 'blue')],
 [('tim', 'red')],
 [('rachel', 'yellow')],
 [('sarah', 'orange'), ('barry', 'green')]]
[[('guido', 'blue')],
 [],
 [],
 [('sarah', 'orange')],
 [],
 [('tim', 'red')],
 [('rachel', 'yellow')],
 [('barry', 'green')]]


In [ ]:
# try dynamic resizing

def resize(buckets, items, n):
    #items = self.items     # save a list of the items
    buckets = [[] for i in range(n)]  # make a new bigger table
    for key, value  in items:   # reinsert saved pairs
        key = value
        
        
# we do cache the hash value
def faster_resize(buckets, items, n):
    #items = self.items     # save a list of the items
    buckets = [[] for i in range(n)]  # make a new bigger table
    for hashvalue, key, value  in items:   # reinsert saved pairs
        key = value
# this is incredibly fast;  almost as fast as a copy


# faster matching
#  in OO - can not assume that __eq__ is fast
def fast_match(key, target_key)
    if key is target_key: return True   # fast
    if key.hash != target_key.hash:   return False   # fast
    return key == target_key    # slow

In [19]:
# next put all the buckets in one table
# using open addressing
def open_addressing_linear(n):
    table = [None] * n
    for h, key, value in entries:
        i = h % n
        while table[i] is not None:
            i = (i + 1) % n
        table[i] = (key, value)
    pprint(table)
open_addressing_linear(8)

[('guido', 'blue'),
 None,
 ('rachel', 'yellow'),
 ('sarah', 'orange'),
 ('tim', 'red'),
 None,
 None,
 ('barry', 'green')]


In [20]:
# deleted entries leaves a whole,  use dummy entries

# Knuth - algorithm D  (1960s)
def lookup(h, key):
    freeslot = None
    for h, key, value in entries:
        i = h % n
        while True:
            entry = table[i]
            if entry == FREE:
                return entry if freeslot is None else freeslot
            elif entry == DUMMY:
                if freeslot is None:
                    freeslot = i  # remember where the dummy is
            elif fast_match(key, entry.key):
                return entry
            i = (i + 1) % n

In [22]:
# multiple hashing
# with linaer probabing we can end up with catastrophic linear pile-up
#  solution is to re-hash on both full values and # of probes

# Tim Peters code
def open_address_multihash(n):
    table = [None] * n
    for h, key, value in entries:
        perturb = h
        i = h % n
        while table[i] is not None:
            print('%r collided with %r' % (key, table[i][0]))
            i = (5 * i + perturb + 1) % n
            perturb >>= 5   # bit shift 5  (use more of the has)
        table[i] = (h, key, value)
    pprint(table)
    
open_address_multihash(8)

'tim' collided with 4293989514786313987
'tim' collided with 4293989514786313987
'tim' collided with 4293989514786313987
'tim' collided with 6432004217604261546
'tim' collided with 3205703081059114744
'tim' collided with 6432004217604261546
[(3205703081059114744, 'guido', 'blue'),
 (3926032199644776547, 'tim', 'red'),
 (6432004217604261546, 'rachel', 'yellow'),
 (4293989514786313987, 'sarah', 'orange'),
 None,
 None,
 None,
 (8459490058338865399, 'barry', 'green')]


In [24]:
# early out for lookups
#  PEP 509 - add previate version to dict  (3.6)   Victor Stinner

# Compact Dict - Raymond Tettinger

def compact_and_ordered(n):
    table = [None] * n
    for pos, entry in enumerate(entries):
        h = perturb = entry[0]
        i = h % n
        while table[i] is not None:
            i = (5 * i + perturb + 1) % n
            perturb >>= 5
        table[i] = pos
    pprint(entries)
    pprint(table)
compact_and_ordered(8)

[(3205703081059114744, 'guido', 'blue'),
 (4293989514786313987, 'sarah', 'orange'),
 (8459490058338865399, 'barry', 'green'),
 (6432004217604261546, 'rachel', 'yellow'),
 (3926032199644776547, 'tim', 'red')]
[0, 4, 3, 1, None, None, None, 2]


In [27]:
# the problem is each dictionary (the one for cities and fruit is shared)
# PEP 412 - Key Sharing solves this

def shared_and_compact(n):
    """ compact, ordered, and shared """
    table = [None] * n
    for pos, entry in enumerate(comb_entries):
        h = perturb = entry[0]
        i = h % n
        while table[i] is not None:
            i = (5 * i + perturb + 1) % n
            perturb >>= 5
        table[i] = pos
    pprint(comb_entries)
    pprint(table)
shared_and_compact(8)            

[(3205703081059114744, 'guido', 'blue', 'austin', 'apple'),
 (4293989514786313987, 'sarah', 'orange', 'denver', 'banna'),
 (8459490058338865399, 'barry', 'green', 'tuscon', 'orange'),
 (6432004217604261546, 'rachel', 'yellow', 'reno', 'pear'),
 (3926032199644776547, 'tim', 'red', 'portland', 'peach')]
[0, 4, 3, 1, None, None, None, 2]


In [ ]:
# future - increase sparsity to reduce collisions
#  can get increased sparsity 

# these are back to databases with index